In [40]:
import sqlite3
import pandas as pd

In [41]:
xls = pd.ExcelFile(r"C:\Users\gamer\OneDrive\Documents\RecommendationEngine\data.xlsx")
df_main = pd.read_excel(xls, 'Orders')
df_returns = pd.read_excel(xls, 'Returns')


In [42]:
df_main.columns = df_main.columns.str.strip()
df_returns.columns = df_returns.columns.str.strip()

In [43]:
connection = sqlite3.connect("demo.db")
cursor = connection.cursor()

In [44]:
df_main.to_sql('superstore', connection, if_exists='replace')
df_returns.to_sql('returns', connection, if_exists='replace')

1634

In [45]:
cursor.execute("""
    ALTER TABLE superstore
    ADD COLUMN returned BOOLEAN
""")

connection.commit()

In [46]:
cursor.execute("""
    UPDATE superstore
    SET returned = 1
    WHERE EXISTS (
        SELECT 1 FROM returns
        WHERE superstore."Order ID" = returns."Order ID"
    )
""")

connection.commit()

In [47]:
cursor.execute("""
    UPDATE superstore
    SET returned = 0
    WHERE returned IS NULL
""")

connection.commit()

In [48]:
df_superstore = pd.read_sql_query("SELECT * FROM superstore", connection)

missing_values = df_superstore.isnull().sum()

print("Missing Values in 'superstore' dataset:")
print(missing_values)


Missing Values in 'superstore' dataset:
index                    0
Row ID                   0
Order Priority           0
Discount                 0
Unit Price               0
Shipping Cost            0
Customer ID              0
Customer Name            0
Ship Mode                0
Customer Segment         0
Product Category         0
Product Sub-Category     0
Product Container        0
Product Name             0
Product Base Margin     72
Region                   0
State or Province        0
City                     0
Postal Code              0
Order Date               0
Ship Date                0
Profit                   0
Quantity ordered new     0
Sales                    0
Order ID                 0
returned                 0
dtype: int64


In [49]:
cursor.execute("DELETE FROM superstore WHERE [Product Base Margin] IS NULL")

connection.commit()

cursor.execute("SELECT COUNT(*) FROM superstore WHERE [Product Base Margin] IS NULL")
missing_count_after_delete = cursor.fetchone()[0]
print("Number of rows with missing 'Product Base Margin' after deletion:", missing_count_after_delete)

Number of rows with missing 'Product Base Margin' after deletion: 0


In [50]:
cursor.execute("""
DELETE FROM superstore
WHERE [Product Name] IS NOT NULL
  AND [Product Name] <> ''
  AND [Product Name] GLOB '[0-9]*';
""")

connection.commit()

In [51]:
query = pd.read_sql_query('SELECT * FROM superstore', connection)
df = pd.DataFrame(query).drop(['index'], axis=1)

df = df.set_index('Row ID')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8914 entries, 18606 to 24492
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Order Priority        8914 non-null   object 
 1   Discount              8914 non-null   float64
 2   Unit Price            8914 non-null   float64
 3   Shipping Cost         8914 non-null   float64
 4   Customer ID           8914 non-null   int64  
 5   Customer Name         8914 non-null   object 
 6   Ship Mode             8914 non-null   object 
 7   Customer Segment      8914 non-null   object 
 8   Product Category      8914 non-null   object 
 9   Product Sub-Category  8914 non-null   object 
 10  Product Container     8914 non-null   object 
 11  Product Name          8914 non-null   object 
 12  Product Base Margin   8914 non-null   float64
 13  Region                8914 non-null   object 
 14  State or Province     8914 non-null   object 
 15  City                 

In [52]:
connection.close()